In [1]:
import sys
molgen_path = "../"
if molgen_path not in sys.path:
    sys.path.insert(0, molgen_path)

In [2]:
from rdkit import Chem
from rdkit.Chem import Mol
from rdkit.Chem import inchi

def is_same_molecule(mol1: Mol, mol2: Mol):
    inchi1 = inchi.MolToInchiKey(mol1)
    inchi2 = inchi.MolToInchiKey(mol2)
    return inchi1 == inchi2

mol1 = Chem.MolFromHELM("PEPTIDE1{A.L.Y.A.S.K.L.S.[am]}$$$$")
mol2 = Chem.MolFromSmiles("CC(C)C[C@H](NC(=O)[C@H](C)N)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](C)C(=O)N[C@@H](CO)C(=O)N[C@@H](CCCCN)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CO)C(N)=O")

print(is_same_molecule(mol1, mol2))

True


In [3]:
#build test data
import csv

test_data: list[tuple] = []
file_path = molgen_path + "data/helm/chembl34_protein_helm.csv"

with open(file_path, newline='', encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=',')
    next(reader)    # skip header
    for row in reader:
        test_data.append(tuple(row))

In [ ]:
#test converter

%load_ext autoreload
%autoreload 2
import cProfile
from utils import MonomersLib, HELMConverter

def test():
    monomers_lib_path = molgen_path + "data/helm/library/chembl_35_monomer_library.xml"
    lib = MonomersLib()
    lib.load_lib(monomers_lib_path)
    converter = HELMConverter(lib)
    success_count = 0
    unmatch_count = 0
    none_count = 0
    for helm, id, smiles in test_data:
        # mol1 = Chem.MolFromHELM(helm)
        mol1 = converter.convert(helm)
        mol2 = Chem.MolFromSmiles(smiles)
        
        # mol3 = Chem.MolFromHELM(helm)
        # if mol3 is None and mol1 is not None and is_same_molecule(mol1, mol2):
        #     print(helm)    

        if mol1 is None:
            none_count += 1
            print("None - helm: " + helm + ", smiles: " + smiles)
        elif is_same_molecule(mol1, mol2):
            success_count += 1
        else:
            unmatch_count += 1
            print("Unmatch - helm: " + helm + ", smiles: " + smiles)
    print("success_count: " + str(success_count) + ", none_count: " + str(none_count) + " unmatch_count: " + str(unmatch_count))

#cProfile.run("test()")
test()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Unmatch - helm: PEPTIDE1{[X1590].L.F}$$$$, smiles: C=C(C)NC(=O)N[C@@H](CCSC)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](Cc1ccccc1)C(=O)O
Unmatch - helm: PEPTIDE1{[Boc_K].[X2362].I.L}$$$$, smiles: CC[C@H](C)[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)N/C=C/C(=O)[C@@H]1CCCN1C(=O)[C@H](CCCCN)NC(=O)OC(C)(C)C)C(=O)N[C@@H](CC(C)C)C(=O)O


[12:32:52] WARNING: not removing hydrogen atom with dummy atom neighbors
[12:32:52] WARNING: not removing hydrogen atom with dummy atom neighbors


Unmatch - helm: PEPTIDE1{G.C.C.S.[X575].[dP].A.C.A.N.N.Q.D.Y.C.[am]}$$$$, smiles: C[C@H](NC(=O)[C@H](CS)NC(=O)[C@H](C)NC(=O)[C@H]1CCCN1C(=O)[C@H](CC1=CNCN1)NC(=O)[C@H](CO)NC(=O)[C@H](CS)NC(=O)[C@H](CS)NC(=O)CN)C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CS)C(N)=O
Unmatch - helm: PEPTIDE1{[ac].[dF].[1-Nal].[X2149]}$$$$, smiles: CC(=O)N[C@H](Cc1ccccc1)C(=O)N[C@@H](Cc1cccc2ccccc12)C(=O)N[C@@H](CCCN=C(N)N)C(=O)c1nc2ccccc2o1.O=C(O)C(F)(F)F
Unmatch - helm: PEPTIDE1{[X1402].[meW].[d2-Pal].[Sar]}$$$$, smiles: CC(C)C[C@@H](NC(=O)N1CCCCCC1)C(=O)N(C)[C@@H](Cc1c[nH]c2ccccc12)C(=O)N[C@H](Cc1ccccn1)C(=O)N(C)CC(=O)[O-].[Na+]
Unmatch - helm: PEPTIDE1{[X1402].[meW].[d2-Pal].F}$$$$, smiles: CC(C)C[C@@H](NC(=O)N1CCCCCC1)C(=O)N(C)[C@@H](Cc1c[nH]c2ccccc12)C(=O)N[C@H](Cc1ccccn1)C(=O)N[C@@H](Cc1ccccc1)C(=O)[O-].[Na+]
Unmatch - helm: PEPTIDE1{R.P.K.[dP].Q.Q.[X11].F.G.L.M.[am]}$$$$, smiles: CSCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)CNC(=O

[12:32:55] Invalid InChI prefix in generating InChI Key


Unmatch - helm: PEPTIDE1{[meL].[meV].[X1670].[Abu].[Sar].[meL].V.[meL].A.[dA].[meL]}$PEPTIDE1,PEPTIDE1,11:R2-1:R1$$$, smiles: C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC)C(=O)N(C)CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](C(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N[C@H](C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N(C)[C@@H](C(C)C)C(=O)N1C
Unmatch - helm: PEPTIDE1{[Hpr].[Me_dF].P.[dW].[dalloI].[dHpr]}$PEPTIDE1,PEPTIDE1,6:R2-1:R1$$$, smiles: CC[C@H](C)[C@H]1NC(=O)[C@@H](Cc2c[nH]c3ccccc23)NC(=O)[C@@H]2CCCN2C(=O)[C@@H](Cc2ccccc2)N(C)C(=O)[C@@H]2CCCCN2C(=O)[C@H]2CCCCN2C1=O
Unmatch - helm: PEPTIDE1{[X1898].[dHpr].H.[dP].W.I}$PEPTIDE1,PEPTIDE1,6:R2-1:R1$$$, smiles: CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2c[nH]c3ccccc23)NC(=O)[C@H]2CCCN2C(=O)[C@H](Cc2c[nH]cn2)NC(=O)[C@H]2CCCCN2C(=O)[C@@H]2CCNCN2C1=O
Unmatch - helm: PEPTIDE1{[X1670].[Abu].[Sar].[meL].V.[meL].A.[X1837].[meL].[meL].[meV]}$PEPTIDE1,PEPTIDE1,11:R2-1:R1$$$, smiles: C/C=C/C[C@@H](C)[C@@H](O)[C@H]1C(=O)N[C@@H](CC)C(=O)N(C)CC(=O)N

...

Unmatch - helm: PEPTIDE1{[dK]}|PEPTIDE2{[ac].[dNal].[X3].[dW].D.E.R.L}|PEPTIDE3{P.[Dpr].[am]}$PEPTIDE1,PEPTIDE3,1:R2-1:R1|PEPTIDE3,PEPTIDE2,2:R3-5:R3|PEPTIDE2,PEPTIDE1,6:R3-1:R3|PEPTIDE2,PEPTIDE1,8:R2-1:R1$$$, smiles: CC(=O)N[C@H](Cc1ccc2ccccc2c1)C(=O)N[C@@H](Cc1ccc(F)cc1)C(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)N[C@H]1CC(=O)NC[C@@H](C(N)=O)NC(=O)[C@@H]2CCCN2C(=O)[C@H]2CCCCNC(=O)CC[C@H](NC1=O)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N[C@@H](CC(C)C)C(=O)N2
Unmatch - helm: PEPTIDE1{G}|PEPTIDE2{[dDab]}|PEPTIDE3{[ac].[dNal].[X3].[d3-Pal].D.E.R.L}|PEPTIDE4{P.[Dpr].[am]}$PEPTIDE2,PEPTIDE4,1:R2-1:R1|PEPTIDE4,PEPTIDE3,2:R3-5:R3|PEPTIDE3,PEPTIDE1,6:R3-1:R1|PEPTIDE2,PEPTIDE1,1:R3-1:R2|PEPTIDE3,PEPTIDE2,8:R2-1:R1$$$, smiles: CC(=O)N[C@H](Cc1ccc2ccccc2c1)C(=O)N[C@@H](Cc1ccc(F)cc1)C(=O)N[C@H](Cc1cccnc1)C(=O)N[C@H]1CC(=O)NC[C@@H](C(N)=O)NC(=O)[C@@H]2CCCN2C(=O)[C@H]2CCNC(=O)CNC(=O)CC[C@H](NC1=O)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N[C@@H](CC(C)C)C(=O)N2
Unmatch - helm: PEPTIDE1{[ac].Y.M.G.W.M.[X451].F.[am]}$$$$, smiles: CSCC[

[12:33:40] Invalid InChI prefix in generating InChI Key


Unmatch - helm: PEPTIDE1{[ac].Y.M.G.W.M.[X453].F.[am]}$$$$, smiles: CSCC[C@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(C)=O)C(=O)NCC(=O)N[C@@H](Cc1c[nH]c2ccccc12)C(=O)N[C@@H](CCSC)C(=O)N1C[C@H](OS(=O)(=O)[O-])C[C@H]1C(=O)N[C@@H](Cc1ccccc1)C(N)=O.[Na+]
Unmatch - helm: PEPTIDE1{[ac].[X96].M.G.W.M.[X453].F.[am]}$$$$, smiles: CSCC[C@H](NC(=O)[C@H](Cc1ccc(OS(=O)(=O)[O-])cc1)NC(C)=O)C(=O)NCC(=O)N[C@@H](Cc1c[nH]c2ccccc12)C(=O)N[C@@H](CCSC)C(=O)N1C[C@H](OS(=O)(=O)[O-])C[C@H]1C(=O)N[C@@H](Cc1ccccc1)C(N)=O.[Na+].[Na+]
Unmatch - helm: PEPTIDE1{[X1821].[X1820].[Sar].[meL].V.[meL].A.[dA].[meL].[meL].[meV]}$PEPTIDE1,PEPTIDE1,11:R2-1:R1$$$, smiles: CCC/C=C(/C)[C@@H](O)[C@H]1C(=O)N[C@@H]([C@@H](C)OCCCO)C(=O)N(C)CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](C(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N[C@H](C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N(C)[C@@H](CC(C)C)C(=O)N(C)[C@@H](C(C)C)C(=O)N1C
Unmatch - helm: PEPTIDE1{[X1761].[Abu].[Sar].[meL].V.[meL].A.[dA].[meL].[meL].[meV]}$PEPTIDE1,PEPTIDE1,11:R2-1:R1$$$, smiles: CC[C@@H]1N

[12:33:50] Invalid InChI prefix in generating InChI Key
[12:33:50] Invalid InChI prefix in generating InChI Key
[12:33:50] Invalid InChI prefix in generating InChI Key


Unmatch - helm: PEPTIDE1{[ac].[Nle].D.H}|PEPTIDE2{[dNal].R.W.K.[am]}|PEPTIDE3{[dNal].R.W.K.P.G.P.G.P.G.P.G.P.G.P.G.[Nle].D.H}$PEPTIDE3,PEPTIDE1,4:R3-3:R3|PEPTIDE3,PEPTIDE2,19:R2-1:R1|PEPTIDE3,PEPTIDE2,18:R3-4:R3|PEPTIDE1,PEPTIDE3,4:R2-1:R1$$$, smiles: CCCC[C@H](NC(C)=O)C(=O)N[C@H]1CC(=O)NCCCC[C@@H](C(=O)N2CCC[C@H]2C(=O)NCC(=O)N2CCC[C@H]2C(=O)NCC(=O)N2CCC[C@H]2C(=O)NCC(=O)N2CCC[C@H]2C(=O)NCC(=O)N2CCC[C@H]2C(=O)NCC(=O)N2CCC[C@H]2C(=O)NCC(=O)N[C@@H](CCCC)C(=O)N[C@H]2CC(=O)NCCCC[C@@H](C(N)=O)NC(=O)[C@H](Cc3c[nH]c4ccccc34)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](Cc3ccc4ccccc4c3)NC(=O)[C@H](Cc3cnc[nH]3)NC2=O)NC(=O)[C@H](Cc2c[nH]c3ccccc23)NC(=O)[C@H](CCCNC(=N)N)NC(=O)[C@@H](Cc2ccc3ccccc3c2)NC(=O)[C@H](Cc2cnc[nH]2)NC1=O
Unmatch - helm: PEPTIDE1{R.W.K.[am]}|PEPTIDE2{[dNal].R.W.K.P.G.P.G.P.G.P.G.P.G.P.G.P.G.P.G.P.G.[Nle].D.H.[dNal]}|PEPTIDE3{[ac].[Nle].D.H}$PEPTIDE3,PEPTIDE2,4:R2-1:R1|PEPTIDE3,PEPTIDE2,3:R3-4:R3|PEPTIDE2,PEPTIDE1,26:R2-1:R1|PEPTIDE2,PEPTIDE1,24:R3-3:R3$$$, smiles: CCCC[C@H](NC(C)=O)C